# 1. Import libraries and load dataset


In [ ]:
import numpy as np 
import pandas as pd 

df_oil = pd.read_csv(r"D:\Topic_13_Project\Topic_13_Retail_Store_Sales_Time_Series\data\raw\oil.csv")

# 2. Display basic information about the dataset


In [3]:
print("First 5 Rows of Data Frame:\n", df_oil.head(5))
print("Data Frame Shape:\n", df_oil.shape)
print("Data Frame Info:\n", df_oil.info())
print("Data Frame Statistics:\n", df_oil.describe())

First 5 Rows of Data Frame:
          date  dcoilwtico
0  2013-01-01         NaN
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20
Data Frame Shape:
 (1218, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB
Data Frame Info:
 None
Data Frame Statistics:
         dcoilwtico
count  1175.000000
mean     67.714366
std      25.630476
min      26.190000
25%      46.405000
50%      53.190000
75%      95.660000
max     110.620000


# 3. Missing Values Summary

In [4]:
# Calculate missing values
missing_counts = df_oil.isnull().sum()
missing_percentage = missing_counts / len(df_oil) * 100

# Create a summary DataFrame
missing_df_oil = pd.DataFrame({
    'Missing Values': missing_counts,
    'Percentage': missing_percentage
})

print("Missing Values Summary:\n", missing_df_oil)

# Filter columns with missing values
missing_df_oil = missing_df_oil[missing_df_oil['Missing Values'] > 0].sort_values(by='Missing Values', ascending=False)

Missing Values Summary:
             Missing Values  Percentage
date                     0    0.000000
dcoilwtico              43    3.530378


# 4. Handling Missing Values

In [5]:
# Handle missing values for oil price (time-series data)
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil = df_oil.sort_values('date')

# Forward fill missing oil prices
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].ffill()

# Backward fill in case missing values exist at the start
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].bfill()

# Recheck missing values after handling
print("Missing Values After Handling:\n", df_oil.isnull().sum())


Missing Values After Handling:
 date          0
dcoilwtico    0
dtype: int64
